# Lambda function 

In [ ]:
import json
import requests
import pandas as pd
from datetime import datetime


def lambda_handler(event, context):
    schema="Gans_support_data"
    host=""
    user="admin"
    password="your_password" #os.environ.get('PASSWORD')
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # TODO implement

    cities = ['Berlin', 'Hannover', 'Bremen', 'Hamburg', 'Munich','Dresden', 'Paris', 'Barcelona', 'Lisbon', 'Brussels', 'Amsterdam', 'Budapest']

    for city in cities:
        city_data = wiki_scrape(city)
        weather_data = get_weather(city)
        
    airport_codes = airport_codes(cities)
    icao_list = airport_codes
    
    flights_df = flight_arrivals(icao_list, flight_API_key)
    
        
    city_data.to_sql('weather', if_exists='append', con=con, index=False)
    flight_df.to_sql('flight', if_exists='append', con=con, index=False)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

def wiki_scrape(city):
    info_list = []


    url = f"https://en.wikipedia.org/wiki/{city}"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    response_dict = {'city': soup.select('#firstHeading')[0].get_text(),
                     'country': soup.select('.infobox-data')[0].get_text(),
                     'latitude': soup.select('.latitude')[0].get_text(),
                     'longitude': soup.select('.longitude')[0].get_text()}

    if soup.select_one('.infobox-label:-soup-contains("Elevation")'):
        response_dict['elevation'] = soup.select_one('.infobox-label:-soup-contains("Elevation")').find_next(
            class_='infobox-data').get_text()
        response_dict['website'] = soup.select_one('.infobox-label:-soup-contains("Website")').find_next(
            class_='infobox-data').get_text()

    if soup.select_one('.infobox-label:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('.infobox-label:-soup-contains("Population")').find_next(
            class_='infobox-data').get_text()

    info_list.append(response_dict)

    cities_info = pd.DataFrame(info_list)

    return cities_info

def get_weather(city):
    weather_dict = {'city': [],
                    'country': [],
                    'forecast_time': [],
                    'outlook': [],
                    'detailed_outlook': [],
                    'temperature': [],
                    'temperature_feels_like': [],
                    'clouds': [],
                    'rain': [],
                    'snow': [],
                    'wind_speed': [],
                    'wind_deg': [],
                    'humidity': [],
                    'pressure': [],
                    'information_retrieved_at': []}

    for city in cities:
        now = datetime.now().astimezone()

        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric" #enter_your_api_key

        response = requests.get(url)
        result = response.json()

        for i in result['list']:
            weather_dict['city'].append(result['city']['name'])
            weather_dict['country'].append(result['city']['country'])
            weather_dict['forecast_time'].append(i['dt_txt'])
            weather_dict['outlook'].append(i['weather'][0]['main'])
            weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
            weather_dict['temperature'].append(i['main']['temp'])
            weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
            weather_dict['clouds'].append(i['clouds']['all'])

            try:
                weather_dict['rain'].append(i['rain']['3h'])
            except:
                weather_dict['rain'].append('0')
            try:
                weather_dict['snow'].append(i['snow']['3h'])
            except:
                weather_dict['snow'].append('0')

                weather_dict['wind_speed'].append(i['wind']['speed'])
                weather_dict['wind_deg'].append(i['wind']['deg'])
                weather_dict['humidity'].append(i['main']['humidity'])
                weather_dict['pressure'].append(i['main']['pressure'])
                weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
    
    weather_df = pd.DataFrame.from_dict(weather_dict, orient='index')
    weather_df = weather_df.transpose()  
    return weather_df

def airport_codes(cities):
    icao_list = []

    url = "https://flightradar24-com.p.rapidapi.com/airports/search"

    ids_list = []
    for city in cities:
        querystring = {"q": city}

        headers = {
            "X-RapidAPI-Key": "API_key", #your-api-key
            "X-RapidAPI-Host": "flightradar24-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        result = response.json()

        for i in result['data']:
            ids_list.append(i['id'])

    # Different API call for airport codes
    for id in ids_list:
        url = "https://flightradar24-com.p.rapidapi.com/airports/detail"

        querystring = {"airport_id": id}

        headers = {
            "X-RapidAPI-Key": API_key, #your-api-key
            "X-RapidAPI-Host": "flightradar24-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        result = response.json()

        if result['data']['airport']['pluginData']['details']['code']['iata'] == id:
            icao_list.append(result['data']['airport']['pluginData']['details']['code']['icao'])
    
    return icao_list

def flight_arrivals(icao_list):
    # Initialize an empty list to store flight data
    list_for_df = []

    # Loop over each ICAO code in the input list
    for icao in icao_list:
        params = {
            'access_key': API_key, #your_api_key
            'arr_icao': icao
        }

        api_result = requests.get('http://api.aviationstack.com/v1/flights', params)

        api_response = api_result.json()

        for flight in api_response['data']:
            flight_dict = {}

            flight_dict['arrival_icao'] = icao
            flight_dict['arrival_time_local'] = datetime.fromisoformat(
            flight['arrival']['scheduled'][:-6]).astimezone(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

            flight_dict['arrival_terminal'] = flight['arrival']['terminal']
            flight_dict['departure_city'] = flight['departure']['timezone']
            flight_dict['departure_icao'] = flight['departure']['icao']
            flight_dict['departure_time_local'] = datetime.fromisoformat(
            flight['departure']['scheduled'][:-6]).astimezone(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

            flight_dict['airline'] = flight['airline']['name']
            flight_dict['flight_number'] = flight['flight']['number']
            flight_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()

            list_for_df.append(flight_dict)

    # Convert the list of flight dictionaries to a DataFrame and return it
    return pd.DataFrame(list_for_df)
